## AI Game playing project

In [ ]:
# prompt: generate chess game

import random,copy
max_move = 15
checkmate_seq = []
class ChessGame:
    def __init__(self, black_king_pos):
        self.board = [['.' for _ in range(8)] for _ in range(8)]
        self.board_black = [['.' for _ in range(8)] for _ in range(8)]
        self.current_player = 'white'
        self.black_king = self.position_to_index(black_king_pos,'add_black_king')
        self.pieces_black = []
        self.pieces_white = []
        self.possible_moves = dict()
        self.possible_moves = self.initiate_possible_moves()
        self.checked = False
        self.stalemated = False
        self.checkmated = False
        self.game_valid = False
        self.score = 0
        self.add_piece('k'+black_king_pos)
        self.move_count = 0
    
    def initiate_possible_moves(self):
        possible_moves = {'white':dict(),'black':dict()}
        for piece in ['K','P','R','N','B','Q']:
            possible_moves['white'][piece] = []
            possible_moves['black'][piece.lower()] = []
        return possible_moves

    def print_board(self):
        print('  +---+---+---+---+---+---+---+---+' + '\t  +---+---+---+---+---+---+---+---+')
        for row in range(8):
            print(8-row, end=' | ')
            for col in range(8):
                print(self.board[8-row-1][col], end=' | ')
            print(f'\t{8-row}', end=' | ')
            for col in range(8):
                print(self.board_black[8-row-1][col], end=' | ')
            print('\n  +---+---+---+---+---+---+---+---+' + '\t  +---+---+---+---+---+---+---+---+')
        print('    a   b   c   d   e   f   g   h'+ '\t    a   b   c   d   e   f   g   h')
    
    def print_possible_moves(self):
        print('White posssible moves: ',self.possible_moves['white'])
        print('Black posssible moves: ',self.possible_moves['black'])
    
    def position_to_index(self, inp, mode):
        if mode == 'add_black_king':
            col = ord(inp[0].lower())-97
            row = int(inp[1])-1
            return tuple([row,col])
        elif mode == 'add':
            piece = inp[0]
            col = ord(inp[1].lower())-97
            row = int(inp[2])-1
            return tuple([piece,row,col])
        else:
            # move
            if len(inp) == 3 or len(inp) == 4:
                # Pe7, Pe8Q
                piece = inp[0]
                col = ord(inp[1].lower())-97
                row = int(inp[2])-1
                moves = []
                promote = '_'
                if len(inp) == 4:
                    promote = inp[3]
                for move in self.possible_moves[self.current_player][piece]:
                    if row == move[2] and col == move[3] and promote == move[4]:
                        moves.append(move)
                if len(moves) ==  0:
                    print('Invalid move')
                elif len(moves) > 1:
                    print('Ambiguous move')
                    for m in moves:
                        print('move: ',tuple([piece]+list(m)))
                else:
                    print('move: ',tuple([piece]+list(moves[0])))
                    return tuple([piece]+list(moves[0]))
                return  None
            else:
                # len 5,6
                # Pe6e7, Pe7e8Q
                piece = inp[0]
                col = ord(inp[1].lower())-97
                row = int(inp[2])-1
                col2 = ord(inp[3].lower())-97
                row2 = int(inp[4])-1
                promote = '_'
                if len(inp) == 6:
                    promote = inp[5]
                print('move: ',tuple([piece,row,col,row2,col2,promote]))
                return tuple([piece,row,col,row2,col2,promote])
        
    def add_piece(self,order):
        piece,row,col = self.position_to_index(order,'add')
        if not self.adjacent_to_black_king(row,col):
            if 0 <= row < 8 and 0 <= col <8 and piece in list(self.possible_moves['white'].keys())+list(self.possible_moves['black'].keys()):
                if piece == 'K' and 'K' in self.pieces_white:
                    print('Only one white king can be added')
                elif piece == 'k' and 'k' in self.pieces_black:
                    print('Only one black king can be added')
                elif self.board[row][col] == '.':
                    self.board[row][col] = piece
                    self.eval_board()
                    if not self.checked:
                        print('Successfully added '+ order + '\n')
                        if piece.islower():
                            self.pieces_black.append(piece)
                        else:
                            if piece == 'B':
                                if row%2 == 0 and col%2 == 0:
                                    piece = 'bB'
                                else:
                                    piece = 'wB'
                            self.pieces_white.append(piece)
                        self.game_valid = self.check_game_valid()
                        return True
                    else:
                        self.board[row][col] = '.'
                        self.eval_board()
                        print('Black cannot be checked at start')
                else:
                    print('Piece already exists at that position')
            else:
                print('Invalid position or piece')
        else:
            print("Cannot place piece adjacent to black king")
        return False
    
    def eval_board(self):
        # {'white':{'P':[[from_row,from_col,to_row,to_col,promote],[]...]}}}
        possible_moves = self.initiate_possible_moves()
        new_black_board = [['.' for _ in range(8)] for _ in range(8)]
        white_pieces = []
        # Mark black king's position
        for row in range(8):
            for col in range(8):
                if self.board[row][col] == 'k':
                    bkr = row
                    bkc = col
                    self.black_king = tuple([row,col])

        for row in range(8):
            for col in range(8):
                if self.board[row][col].isupper():
                    piece = self.board[row][col]
                    if piece == 'B':
                        if row%2 == 0 and col%2 == 0:
                            piece = 'bB'
                        else:
                            piece = 'wB'
                    white_pieces.append(piece) 

                if self.board[row][col] == '.':
                    continue

                elif self.board[row][col] == 'P':
                    # Pawn
                    if 0 <= row + 1 < 7:
                        if self.board[row+1][col] == '.':
                            possible_moves['white']['P'].append(tuple([row,col,row+1,col,'_']))
                        if 0 <= col + 1 < 8 and self.board[row+1][col+1].islower() :
                            possible_moves['white']['P'].append(tuple([row,col,row+1,col+1,'_']))
                        if 0 <= col - 1 < 8 and self.board[row+1][col-1].islower():
                            possible_moves['white']['P'].append(tuple([row,col,row+1,col-1,'_']))
                    elif row + 1 == 7:
                        #promote
                        if self.board[row+1][col] == '.':
                            possible_moves['white']['P'].append(tuple([row,col,row+1,col,'N']))
                            possible_moves['white']['P'].append(tuple([row,col,row+1,col,'Q']))
                        if col + 1 < 8 and self.board[row+1][col+1].islower():
                            possible_moves['white']['P'].append(tuple([row,col,row+1,col+1,'N']))
                            possible_moves['white']['P'].append(tuple([row,col,row+1,col+1,'Q']))
                        if 0 <= col - 1 and self.board[row+1][col-1].islower():
                            possible_moves['white']['P'].append(tuple([row,col,row+1,col-1,'N']))
                            possible_moves['white']['P'].append(tuple([row,col,row+1,col-1,'Q']))
                    if 0 <= row + 1 < 8:
                        if col + 1 < 8:
                            new_black_board[row+1][col+1] = 'x'
                        if col - 1 >= 0:
                            new_black_board[row+1][col-1] = 'x'

                elif self.board[row][col] == 'R':
                    # Rook
                    for dir in [1,-1]:
                        i = 1
                        n_row = row + i*dir
                        n_col = col
                        while 0 <= n_row < 8 and 0 <= n_col < 8:
                            new_black_board[n_row][n_col] = 'x'
                            if not self.board[n_row][n_col].isupper():
                                possible_moves['white']['R'].append(tuple([row,col,n_row,n_col,'_']))
                            else:
                                break
                            i += 1
                            n_row = row + i*dir
                            n_col = col
                        i = 1
                        n_row = row
                        n_col = col + i*dir
                        while 0 <= n_row < 8 and 0 <= n_col < 8:
                            new_black_board[n_row][n_col] = 'x'
                            if not self.board[n_row][n_col].isupper():
                                possible_moves['white']['R'].append(tuple([row,col,n_row,n_col,'_']))
                            else:
                                break
                            i += 1
                            n_row = row
                            n_col = col + i*dir

                elif self.board[row][col] == 'N':
                    moveset = [[_,__] for _ in [2,1,-1,-2] for __ in [2,1,-1,-2] if abs(_)!=abs(__)]
                    for m in moveset:
                        n_row = row + m[0]
                        n_col = col + m[1]
                        if 0 <= n_row < 8 and 0 <= n_col < 8:
                            new_black_board[n_row][n_col] = 'x'
                            if not self.board[n_row][n_col].isupper():
                                possible_moves['white']['N'].append(tuple([row,col,n_row,n_col,'_']))

                elif self.board[row][col] == 'B':
                    for dir in [1,-1]:
                        i = 1
                        n_row = row + i*dir
                        n_col = col + i*dir
                        while 0 <= n_row < 8 and 0 <= n_col < 8:
                            new_black_board[n_row][n_col] = 'x'
                            if not self.board[n_row][n_col].isupper():
                                possible_moves['white']['B'].append(tuple([row,col,n_row,n_col,'_']))
                            else:
                                break
                            i += 1
                            n_row = row + i*dir
                            n_col = col + i*dir
                        i = 1
                        n_row = row + i*dir
                        n_col = col - i*dir
                        while 0 <= n_row < 8 and 0 <= n_col < 8:
                            new_black_board[n_row][n_col] = 'x'
                            if not self.board[n_row][n_col].isupper():
                                possible_moves['white']['B'].append(tuple([row,col,n_row,n_col,'_']))
                            else:
                                break
                            i += 1
                            n_row = row + i*dir
                            n_col = col - i*dir

                elif self.board[row][col] == 'Q':
                    for dir in [1,-1]:
                        i = 1
                        n_row = row + i*dir
                        n_col = col
                        while 0 <= n_row < 8 and 0 <= n_col < 8:
                            new_black_board[n_row][n_col] = 'x'
                            if not self.board[n_row][n_col].isupper():
                                possible_moves['white']['Q'].append(tuple([row,col,n_row,n_col,'_']))
                            else:
                                break
                            i += 1
                            n_row = row + i*dir
                            n_col = col
                        i = 1
                        n_row = row
                        n_col = col + i*dir
                        while 0 <= n_row < 8 and 0 <= n_col < 8:
                            new_black_board[n_row][n_col] = 'x'
                            if not self.board[n_row][n_col].isupper():
                                possible_moves['white']['Q'].append(tuple([row,col,n_row,n_col,'_']))
                            else:
                                break
                            i += 1
                            n_row = row
                            n_col = col + i*dir
                        i = 1
                        n_row = row + i*dir
                        n_col = col + i*dir
                        while 0 <= n_row < 8 and 0 <= n_col < 8:
                            new_black_board[n_row][n_col] = 'x'
                            if not self.board[n_row][n_col].isupper():
                                possible_moves['white']['Q'].append(tuple([row,col,n_row,n_col,'_']))
                            else:
                                break
                            i += 1
                            n_row = row + i*dir
                            n_col = col + i*dir
                        i = 1
                        n_row = row + i*dir
                        n_col = col - i*dir
                        while 0 <= n_row < 8 and 0 <= n_col < 8:
                            new_black_board[n_row][n_col] = 'x'
                            if not self.board[n_row][n_col].isupper():
                                possible_moves['white']['Q'].append(tuple([row,col,n_row,n_col,'_']))
                            else:
                                break
                            i += 1
                            n_row = row + i*dir
                            n_col = col - i*dir

                elif self.board[row][col] == 'K':
                    moveset = [[_,__] for _ in [1,0,-1] for __ in [1,0,-1] if _ != 0 or __ != 0]
                    for m in moveset:
                        n_row = row + m[0]
                        n_col = col + m[1]
                        if 0 <= n_row < 8 and 0 <= n_col < 8:
                            new_black_board[n_row][n_col] = 'x'
                            if not self.board[n_row][n_col].isupper() and not self.adjacent_to_black_king(n_row,n_col):
                                possible_moves['white']['K'].append(tuple([row,col,n_row,n_col,'_']))

        # update black possible moves
        moveset = [[_,__] for _ in [1,0,-1] for __ in [1,0,-1] if _ != 0 or __ != 0]
        for m in moveset:
            n_row = bkr + m[0]
            n_col = bkc + m[1]
            if 0 <= n_row < 8 and 0 <= n_col < 8:
                if not new_black_board[n_row][n_col].isupper() and new_black_board[n_row][n_col] != 'x':
                    possible_moves['black']['k'].append(tuple([bkr,bkc,n_row,n_col,'_']))
        # update gamestate
        self.possible_moves = possible_moves
        self.board_black = new_black_board
        self.checked = self.board_black[self.black_king[0]][self.black_king[1]] == 'x'
        if self.checked:
            self.board_black[self.black_king[0]][self.black_king[1]] = '*'
        else:
            self.board_black[self.black_king[0]][self.black_king[1]] = 'k'
        if not self.checked and len(possible_moves['black']['k']) == 0 and self.current_player == 'black':
            self.score = -10000
        else:
            self.score = 8 - len(possible_moves['black']['k'])
        self.pieces_white = white_pieces
        self.stalemated = not self.checked and len(possible_moves['black']['k']) == 0
        self.checkmated = self.checked and len(possible_moves['black']['k']) == 0
        self.game_valid = self.check_game_valid()


    def adjacent_to_black_king(self,row,col):
        bkr,bkc = self.black_king
        moveset = [[bkr+_,bkc+__] for _ in [1,0,-1] for __ in [1,0,-1] if _ != 0 or __ != 0]
        
        return [row,col] in moveset
        
    def check_game_valid(self):
        piece_count = {'P':0,'bB':0,'wB':0,'R':0,'N':0,'Q':0,'K':0}
        for p in self.pieces_white:
            piece_count[p] += 1
        if piece_count['K'] < 1 or self.stalemated or self.checkmated or self.move_count >= max_move:
            return False
        if piece_count['P'] < 1:
            if piece_count['Q'] < 1:
                if piece_count['R'] < 1:
                    if piece_count['N'] < 1:
                        if piece_count['wB'] < 1 or piece_count['bB'] < 1:
                            return False
                    else:
                        if piece_count['bB'] < 1 and piece_count['wB'] < 1:
                            return False
        return True
    
    def make_move(self, order: str):
        t = self.position_to_index(order,'move')
        if t == None:
            print('Invalid move, please try again')
            return
        else:
            self.make_move(t)

    def make_move(self, order: list):
        if order[0].islower() and self.current_player == 'white':
            print('White\'s turn')
            return
        elif order[0].isupper() and self.current_player == 'black':
            print('Black\'s turn')
            return
        piece, from_row, from_col, to_row, to_col, promote = order
        if piece in self.possible_moves[self.current_player].keys():
            if tuple([from_row,from_col,to_row,to_col,promote]) in self.possible_moves[self.current_player][piece]:
                self.board[from_row][from_col] = '.'
                if promote == '_':
                    self.board[to_row][to_col] = piece
                else:
                    self.board[to_row][to_col] = promote
                if self.current_player == 'white':
                    if order[0] == 'P':
                        self.move_count == 0
                    else:
                        self.move_count += 1
                    self.current_player = 'black'
                else:
                    self.current_player = 'white'
        self.eval_board()


def get_random_position(minrow=1,maxrow=8):
    return random.choice([chr(a) for a in range(ord('a'),ord('i'))]) + str(random.choice(range(minrow,maxrow+1)))

def create_random_board():
    piece = 'k'
    pos = get_random_position()
    game = ChessGame(pos)
    added = False
    while not added:
        piece = 'K'
        pos = get_random_position()
        added = game.add_piece(piece + pos)
    start = False
    while not start:    
        piece = random.choice(['P','R','N','B','Q'])
        if piece == 'P':
            pos = get_random_position(2,7)
        else:
            pos = get_random_position()
        print('Adding: ',piece+pos)
        game.add_piece(piece+pos)
        start = game.game_valid
    return game
        
def initiate_board(mode):
    if mode == 'manual':
        # bk = input('Pick black king\'s position')
        game = ChessGame('c6')
        game.add_piece('Ke6')
        game.add_piece('Qd3')
        # game.add_piece('Re4')
    else:
        game = create_random_board()
    return game

def minimax (g: ChessGame, alpha, beta, move = []):
    game = copy.deepcopy(g)
    if len(move) > 0:
        game.make_move(move)
    if not game.game_valid:
        return [game.score, [move]]
    
    if game.current_player == 'white':
        max_score = [-20000,[]]
        for p in game.possible_moves[game.current_player].keys():
            if len(game.possible_moves[game.current_player][p]) != 0:
                for m in game.possible_moves[game.current_player][p]:
                    next_move = [p] + list(m)
                    s = minimax(game,alpha,beta,next_move)
                    max_score = max(max_score,s)
                    alpha = max(alpha,s)
                    if beta[0] <= alpha[0]:
                        break
        best_move = max_score
    elif game.current_player == 'black':
        min_score = [10,[]]
        for p in game.possible_moves[game.current_player].keys():
            if len(game.possible_moves[game.current_player][p]) != 0:
                for m in game.possible_moves[game.current_player][p]:
                    next_move = [p] + list(m)
                    s = minimax(game,alpha,beta,next_move)
                    min_score = min(min_score,s)
                    beta = min(beta,s)
                    if beta[0] <= alpha[0]:
                        break
        best_move = min_score
    if len(move) > 0:
        best_move[1] = [move] + best_move[1]
    return best_move

def main():
    mode = 'manual'
    global max_move
    # mode = input('Select mode (random/manual): ')
    game = initiate_board(mode)
    print("--------------- Board Initiated ---------------")
    game.print_board()
    for i in range(1,15):
        max_move = i 
        max_score = minimax(game)
        if max_score[0] == 8:
            break
        print(f'Max Score for {max_move} move is: {max_score[0]}')
    print("---------------- Calculation Finished ----------------")
    print("minimum move is: ", max_move)
    print("seq: ", max_score[1])
    for m in max_score[1]:
        game.make_move(m)
        print(m)
        game.print_board()


if __name__ == "__main__":
    main()

Successfully added kc6

Successfully added Ke6

Successfully added Qd3

--------------- Board Initiated ---------------
  +---+---+---+---+---+---+---+---+	  +---+---+---+---+---+---+---+---+
8 | . | . | . | . | . | . | . | . | 	8 | . | . | . | x | . | . | . | . | 
  +---+---+---+---+---+---+---+---+	  +---+---+---+---+---+---+---+---+
7 | . | . | . | . | . | . | . | . | 	7 | . | . | . | x | x | x | . | x | 
  +---+---+---+---+---+---+---+---+	  +---+---+---+---+---+---+---+---+
6 | . | . | k | . | K | . | . | . | 	6 | x | . | k | x | . | x | x | . | 
  +---+---+---+---+---+---+---+---+	  +---+---+---+---+---+---+---+---+
5 | . | . | . | . | . | . | . | . | 	5 | . | x | . | x | x | x | . | . | 
  +---+---+---+---+---+---+---+---+	  +---+---+---+---+---+---+---+---+
4 | . | . | . | . | . | . | . | . | 	4 | . | . | x | x | x | . | . | . | 
  +---+---+---+---+---+---+---+---+	  +---+---+---+---+---+---+---+---+
3 | . | . | . | Q | . | . | . | . | 	3 | x | x | x | . | x | x | x | x | 
  +-

KeyboardInterrupt: 

In [44]:
a = [1,2]
b = copy.deepcopy(a)
a[1] = 3
print(b)

[1, 2]
